In [ ]:
import pandas as pd
from IDLPPBopt import PPBPredictor, PPBPredictorViz
from IDLPPBopt.interface.Visualizer import AtomAttentionVisualizer

# Simple prediction

In [ ]:
# ::: READ DATA
#
INPUT_FILE = 'input_compounds.csv'
SMILES_COLUMN = 'cano_smiles'

df = pd.read_csv(INPUT_FILE)
smiles_series = df['cano_smiles']

# ::: RUN PREDICTOR
#
ppbpred = PPBPredictor()
ppbpred.load_smiles(smiles_series)
ppbpred.prepare_model()
results = ppbpred.evaluate()

# ::: SAFE results
#
results = pd.Series(results, index=smiles_series.index)
df['IDL-PPBopt'] = results
df
# df.to_csv('results.csv')

# Visualize prediction

In [ ]:
from IPython.display import	SVG, display

# ::: READ DATA
#
INPUT_FILE = 'input_compounds.csv'
SMILES_COLUMN = 'cano_smiles'

df = pd.read_csv(INPUT_FILE)
smiles_series = df['cano_smiles']

# ::: RUN PREDICTOR
#
ppbviz = PPBPredictorViz()
ppbviz.load_smiles(smiles_series)
ppbviz.prepare_model()
mol_predictions, mol_attnweights = ppbviz.evaluate()

# ::: VISUALIZE PREDICTION
#
for smi, weights, pred in zip(ppbviz.smiles_series, mol_attnweights, mol_predictions):
    # Initializing visualizer
    viz = AtomAttentionVisualizer(smi, weights, vmin=0, vmax=weights.max()*1.25)
    # Higlight privileged substructures
    viz.highlight_privileged_substructures()

    print(smi, pred, sep='\t')
    print(f'Detected {len(viz.substructures):d} privileged substructures...')

    # Generate and display SVG
    svg = viz.draw_svg()
    display(SVG(svg))
